In [1]:
import os 
import ipyvolume as ipv
from tyssue import HistoryHdf5
from tyssue.draw import browse_history
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev
from scipy.optimize import curve_fit

from polarity.toolbox import open_sheet
import seaborn as sns
import scipy.stats as stat
from scipy import stats

In [2]:
from scipy.signal import find_peaks
from scipy.optimize import brentq

In [3]:
def find_max_depth_time(history, window):
    with pd.HDFStore(history.hf5file, "r") as file:
        v_pos = file.select("vert", columns=["time", "x", "y", "z"]).reset_index()
    center = v_pos[np.abs(v_pos['z']) < window]
    center['distance'] = np.sqrt((center.x-center.x.mean())**2+(center.y-center.y.mean())**2)
    distances = center.groupby('time')['distance'].mean()
    time = distances.idxmin()
    return time

In [4]:
def update_latrad(sheet):
    radius_ = sheet.vert_df.eval("(x**2+y**2)**0.5")
    sheet.vert_df['depth'] = radius_.max() - radius_
    sheet.vert_df['latitude'] = np.arctan2(sheet.vert_df.y, sheet.vert_df.x)

def get_slice(sheet, l0=0, largeur=0.3):
    """ l0 should be between -π and π
    
    """
    vert = sheet.vert_df[
        (sheet.vert_df.latitude<l0+largeur/2)
        & (sheet.vert_df.latitude>l0-largeur/2)
        & (sheet.vert_df.z>-30) & (sheet.vert_df.z<30)
    ].copy()
    return vert
    
def measure_fold(sheet, l0=0.0, largeur=0.3, t=None, window=40, l0_to_measure=None, pos='left'):
    
    if t is None:
        t = np.arange(-window/4, window/4+1, window/8)
    vert = get_slice(sheet, l0, largeur)
    vert.sort_values('z', inplace=True)
    tck = splrep(vert['z'], vert['depth'], t=t, k=3)
    zz = np.linspace(-window/2, window/2, 100)
    i_depth = splev(zz, tck)
    i_slope = splev(zz, tck, der=1)
    
    z_sl = zz[np.argmax(i_slope)]
    z_sr = zz[np.argmin(i_slope)]
    try:
        z_depth = brentq(lambda z: splev(z, tck, der=1), z_sl, z_sr)
    except ValueError:
        z_depth = np.nan
        
    if l0_to_measure is None:
        i_depth_max = i_depth.max()
    else: 
        if l0_to_measure != l0:
            i_depth_max = i_depth.max()
        else:
            if pos=='left':
                i_depth_max = i_depth[:int(len(i_depth)/2)].max()
            else:
                i_depth_max = i_depth[int(len(i_depth)/2):].max()
        
    measures = {
        "i_depth":i_depth,
        "depth" : i_depth_max,
        "z_depth": z_depth,
        "slope_left" : i_slope.max(), 
        "slope_right" : -i_slope.min(),
        "z_sl" : z_sl,
        "z_sr" : z_sr,
        "x" : np.mean(vert.x),
    }
    
    
    return measures

def measure_fold_whole_sheet(sheet, nbins=100, t=None, **kwargs):
    update_latrad(sheet)
    all_measures = []
    lats = np.linspace(-np.pi, np.pi, nbins)
    if t is None : 
        t=np.nan
    for lat in lats:
        m = measure_fold(sheet, l0=lat, **kwargs)
        m['t'] = t
        all_measures.append(m)
    
    all_measures = pd.DataFrame.from_records(all_measures)
    all_measures.index = lats
    return all_measures



In [28]:
SIM_DIR = '/media/admin-suz/Sophie/3_Papiers-Polarity-Manu/10_Revisions/20200205-variability'

list_dir = os.listdir(SIM_DIR)

In [ ]:
## 1-Faire la mesure de tout les controles (= sans perturbation)
datas = pd.DataFrame()

for d in range(0,12,2):
    for i in range(0,60):
        for f in (f for f in os.listdir(os.path.join(SIM_DIR,list_dir[d])) if f.split('_')[-1] == str(i)):

            
            history = HistoryHdf5.from_archive(
                os.path.join(os.path.join(SIM_DIR,list_dir[d]),f+'/out.hf5')
                )
            t = find_max_depth_time(history, 15)
            
            print(d, i, t)
            sheet = history.retrieve(t)
            
            all_measures = measure_fold_whole_sheet(sheet, nbins=24, window=40)
            all_measures.reset_index(drop=False, inplace=True)
            all_measures['pola'] = f.split('_')[0]
            all_measures['perturb'] = f.split('_')[2]
            all_measures['id'] = f.split('_')[-1]
             
                
            a = [all_measures.x.min(), 
                 all_measures.z_depth[all_measures.x.idxmin()]]
            if all_measures.z_depth.max()>abs(all_measures.z_depth.min()):
                all_measures['distance']=all_measures.z_depth.max()
                b = [all_measures.x[all_measures.z_depth.idxmax()],
                     all_measures.z_depth.max()]
            else : 
                all_measures['distance']=all_measures.z_depth.min()
                b = [all_measures.x[all_measures.z_depth.idxmin()],
                     all_measures.z_depth.min()]
            

            slope = (a[1]-b[1])/(a[0]-b[0])
            ordo = b[1]-(slope*b[0])
            angle = np.arctan(slope)*180/np.pi
            
            
            #position du pli predictif
            model = np.polyfit(sheet.face_df[sheet.face_df.fold==1].x, sheet.face_df[sheet.face_df.fold==1].z, deg=1)
            angle_origin = np.arctan(model[0])*180/np.pi

            all_measures['angle'] = np.diff((angle_origin,angle))[0]
            
            
            
 
            
            
datas.to_csv("220920_datas_orientation_control_v2.csv", sep=';')
data_save = datas.copy()

/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0 0 53.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0 1 53.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


...

6 11 64.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


6 12 64.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


6 13 63.0


In [ ]:
result_control = pd.DataFrame(columns=['pola', 'perturb', 'id','angle','distance'])

i=0
for id_ in range(0, 60):   
    for pola_ in [ 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,]:
        max_depth_control = [pola_, '-1', id_]
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb=='-1')].index
        max_depth_control.append(datas.loc[index_, 'angle'].max())
        max_depth_control.append((datas.loc[index_, 'distance'].max()))

        result_control.loc[i] = max_depth_control
        i+=1
        
    result_control.append(max_depth_control)
    
datas_control

In [7]:
## 2-Faire la mesure de tout les perturbés avec une polarité de 1
datas = data_save.copy()
d=11
pos_max_05 = []
for i in range(0,60):
    for f in (f for f in os.listdir(os.path.join(SIM_DIR,list_dir[d])) if f.split('_')[-1] == str(i)):

        if f.split('_')[0]=='1.0':
            history = HistoryHdf5.from_archive(
                os.path.join(os.path.join(SIM_DIR,list_dir[d]),f+'/out.hf5')
                )
            t = find_max_depth_time(history, 15)

            print(d, i, t)
            sheet = history.retrieve(t)

            all_measures = measure_fold_whole_sheet(sheet, nbins=24, window=40)
            all_measures.reset_index(drop=False, inplace=True)
            all_measures['pola'] = f.split('_')[0]
            all_measures['perturb'] = f.split('_')[2]
            all_measures['id'] = f.split('_')[-1]
            
            a = [all_measures.x.min(), 
                 all_measures.z_depth[all_measures.x.idxmin()]]
            if all_measures.z_depth.max()>abs(all_measures.z_depth.min()):
                all_measures['distance']=all_measures.z_depth.max()
                b = [all_measures.x[all_measures.z_depth.idxmax()],
                     all_measures.z_depth.max()]
            else : 
                all_measures['distance']=all_measures.z_depth.min()
                b = [all_measures.x[all_measures.z_depth.idxmin()],
                     all_measures.z_depth.min()]
            

            slope = (a[1]-b[1])/(a[0]-b[0])
            ordo = b[1]-(slope*b[0])
            angle = np.arctan(slope)*180/np.pi+90
            
            
            #position du pli predictif
            model = np.polyfit(sheet.face_df[sheet.face_df.fold==1].x, sheet.face_df[sheet.face_df.fold==1].z, deg=1)
            angle_origin = np.arctan(model[0])*180/np.pi+90

            all_measures['angle'] = np.diff((angle_origin,angle))[0]+90
            
            
            datas=pd.concat([datas, all_measures], axis=0)

            fold = all_measures['z_depth']
            if abs(fold.min()) > abs(fold.max()):
                pos_max_05.append(list(fold).index(np.nanmin(fold)))
            else: 
                pos_max_05.append(list(fold).index(np.nanmax(fold)))
            
datas.to_csv("220920_datas_orientation_control_perturb1.csv", sep=';')
data_save2 = datas.copy()

/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


11 0 70.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


11 1 60.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


...

11 57 61.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


11 58 56.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


11 59 71.0


In [8]:
datas = data_save2.copy()

lats = np.linspace(-np.pi, np.pi, 24)
for d in range(1,11,2):
    for i in range(0,60):
        for f in (f for f in os.listdir(os.path.join(SIM_DIR,list_dir[d])) if f.split('_')[-1] == str(i)):

            
            history = HistoryHdf5.from_archive(
                os.path.join(os.path.join(SIM_DIR,list_dir[d]),f+'/out.hf5')
                )
            t = find_max_depth_time(history, 15)
            
            print(d, i, t)
            sheet = history.retrieve(t)
            
            if pos_max_05[i]>12:
                pos_ = 'right'
            else :
                pos_ ='left'
            
            all_measures = measure_fold_whole_sheet(sheet, nbins=24, window=40, 
                                                    l0_to_measure=lats[pos_max_05[i]],
                                                    pos=pos_)
            all_measures.reset_index(drop=False, inplace=True)
            all_measures['pola'] = f.split('_')[0]
            all_measures['perturb'] = f.split('_')[2]
            all_measures['id'] = f.split('_')[-1]
            
            a = [all_measures.x.min(), 
                 all_measures.z_depth[all_measures.x.idxmin()]]
            if all_measures.z_depth.max()>abs(all_measures.z_depth.min()):
                all_measures['distance']=all_measures.z_depth.max()
                b = [all_measures.x[all_measures.z_depth.idxmax()],
                     all_measures.z_depth.max()]
            else : 
                all_measures['distance']=all_measures.z_depth.min()
                b = [all_measures.x[all_measures.z_depth.idxmin()],
                     all_measures.z_depth.min()]
            

            slope = (a[1]-b[1])/(a[0]-b[0])
            ordo = b[1]-(slope*b[0])
            angle = np.arctan(slope)*180/np.pi+90
            
            
            #position du pli predictif
            model = np.polyfit(sheet.face_df[sheet.face_df.fold==1].x, sheet.face_df[sheet.face_df.fold==1].z, deg=1)
            angle_origin = np.arctan(model[0])*180/np.pi+90

            all_measures['angle'] = np.diff((angle_origin,angle))[0]+90
            
            
            datas=pd.concat([datas, all_measures], axis=0)
                        

datas.to_csv("220920_datas_orientation_control_perturb.csv", sep=';')
data_save3 = datas.copy()

/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


1 0 52.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


1 1 49.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


...

9 57 55.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


9 58 52.0


/home/admin-suz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


9 59 67.0


In [9]:
dvsvsdv

NameError: name 'dvsvsdv' is not defined

In [21]:
datas = pd.read_csv('210920_datas_orientation_control_perturb.csv', sep=';', index_col=0)
datas.reset_index(inplace=True)

result_control = pd.DataFrame(columns=['pola', 'perturb', 'id','angle','distance'])

i=0
for id_ in range(0, 60):   
    for pola_ in [ 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,]:
        max_depth_control = [pola_, '-1', id_]
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb=='-1')].index
        max_depth_control.append(datas.loc[index_, 'angle'].max())
        max_depth_control.append((datas.loc[index_, 'distance'].max()))

        result_control.loc[i] = max_depth_control
        i+=1
        
    result_control.append(max_depth_control)
        
result_control.to_csv("220920_control.csv", sep=';')

In [22]:
datas = pd.read_csv('datas_orientation3.csv', sep=';', index_col=0)
datas.reset_index(inplace=True)

result_perturb = pd.DataFrame(columns=['pola', 'perturb', 'id','angle','distance'])

i=0
for id_ in range(0, 60):   
    for pola_ in [ 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,]:
        max_depth_perturb = [pola_, '1', id_]
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb!='-1')].index
        max_depth_perturb.append(datas.loc[index_, 'angle'].max())
        max_depth_perturb.append(datas.loc[index_, 'distance'].max())

        result_perturb.loc[i] = max_depth_perturb
        i+=1
        
        
result_perturb
result_perturb.to_csv("220920_perturb.csv", sep=';')

In [ ]:
datas = pd.read_csv('210920_datas_orientation_control_perturb.csv', sep=';', index_col=0)
datas.reset_index(inplace=True)
colors=['red', 'blue', 'green', 'orange', 'purple', 'black']

result_control = []
result_perturb = []
for id_ in range(0, 60):   
    max_depth_control = []
    max_depth_perturb = []
    i=0
    for pola_ in [ 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,]:
            
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb=='-1')].index
        max_depth_control.append(datas.loc[index_, 'angle'].unique()[0])
            
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb!='-1')].index

        max_depth_perturb.append(datas.loc[index_, 'angle'].unique()[0])
         
        i+=1
    result_control.append(max_depth_control)
    result_perturb.append(max_depth_perturb)
        
result_control = pd.DataFrame(np.array(result_control))
result_control.columns=['0.5', '0.6','0.7','0.8','0.9','1.0']
result_perturb = pd.DataFrame(np.array(result_perturb))
result_perturb.columns=['0.5', '0.6','0.7','0.8','0.9','1.0']


In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(result_control), jitter=True, alpha=0.8, ax=ax)
g = sns.stripplot(data=(result_perturb), jitter=True, alpha=0.2, ax=ax)
# g.set_ylim([70,110])

In [ ]:
datas = pd.read_csv('210920_datas_orientation_control_perturb.csv', sep=';', index_col=0)
datas.reset_index(inplace=True)
colors=['red', 'blue', 'green', 'orange', 'purple', 'black']

result_control = []
result_perturb = []
for id_ in range(0, 60):   
    max_depth_control = []
    max_depth_perturb = []
    i=0
    for pola_ in [ 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,]:
            
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb=='-1')].index
        max_depth_control.append(datas.loc[index_, 'distance'].unique()[0])
            
        index_ = datas[(datas.id==id_)
                     & (datas.pola==pola_)
                     & (datas.perturb!='-1')].index

        max_depth_perturb.append(datas.loc[index_, 'distance'].unique()[0])
         
        i+=1
    result_control.append(max_depth_control)
    result_perturb.append(max_depth_perturb)
        
result_control = pd.DataFrame(np.array(result_control))
result_control.columns=['0.5', '0.6','0.7','0.8','0.9','1.0']
result_perturb = pd.DataFrame(np.array(result_perturb))
result_perturb.columns=['0.5', '0.6','0.7','0.8','0.9','1.0']


In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(result_control), jitter=True, alpha=0.8, ax=ax)
g = sns.stripplot(data=(result_perturb), jitter=True, alpha=0.2, ax=ax)
# g.set_ylim([70,110])

In [ ]:
fgdgre

In [ ]:
# result_control.drop([1,3,5,27,19,50,52,56], axis=0, inplace=True)
# result_perturb.drop([1,3,5,27,19,50,52,56], axis=0, inplace=True)
result_perturb.drop([1,5,56], axis=0, inplace=True)
result_control.drop([1,5,56], axis=0, inplace=True)

In [ ]:
print(stat.levene(result_perturb['0.5'], result_perturb['1.0']))

In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(result_perturb['0.5'], result_perturb['1.0']), jitter=True, ax=ax)
# g.set_ylim([70,110])


In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(result_control['1.0'], result_perturb['1.0']), jitter=True, ax=ax)
g.set_ylim([70,110])


In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(result_control), jitter=True, alpha=0.8, ax=ax)
g = sns.stripplot(data=(result_perturb), jitter=True, alpha=0.2, ax=ax)
# g.set_ylim([70,110])

In [ ]:
result_perturb.boxplot(grid=False,)

In [ ]:
diff_=result_control - result_perturb + 90
fig, ax = plt.subplots()
g = sns.stripplot(data=(diff_), jitter=True,ax=ax)
print(stat.levene(diff_['0.5'], diff_['1.0']))

In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(diff_['0.5'], diff_['1.0']), jitter=True,ax=ax)
print(stat.levene(diff_['0.5'], diff_['1.0']))

In [ ]:
fig, ax = plt.subplots()
g = sns.stripplot(data=(result_control['1.0'], diff_['1.0']), jitter=True,ax=ax)
print(stat.levene(result_control['1.0'], diff_['1.0']))

## Position Perturbation

In [ ]:
# liste fichiers perturbation
SIM_DIR = '/media/admin-suz/Sophie/3_Papiers-Polarity-Manu/10_Revisions/20200205-variability'
list_dir = os.listdir(SIM_DIR)
d = list_dir[1]

list_perturb = []
list_simu=os.listdir(os.path.join(SIM_DIR,d))

for f in list_simu: 
    
    history = HistoryHdf5.from_archive(
                os.path.join(os.path.join(SIM_DIR,d),f+'/out.hf5')
                )
    sheet = history.retrieve(0)
    
    lp=sheet.face_df[sheet.face_df.is_mesoderm == 1].index
    for p in lp:
        list_perturb.append(p)
    

In [ ]:
##Mise à plat du tissu
from math import atan2
from matplotlib.patches import Polygon, FancyArrow, Arc, PathPatch
from matplotlib.collections import PatchCollection, PolyCollection

from tyssue.draw.plt_draw import parse_edge_specs, parse_face_specs
fig = plt.figure()
ax = plt.axes()


sheet = sheet.extract_bounding_box(z_boundary=(-50,50))
#cmap = matplotlib.colors.ListedColormap(['white', 'magenta', 'magenta', 'magenta'])#, 'moccasin'])

#theta = [atan2(sheet.vert_df.y[i], sheet.vert_df.x[i]) for i in range(0, sheet.Nv)]

l_z=[]
l_rt=[]
for idx, edge in sheet.edge_df.iterrows():
    srce = int(edge["srce"])
    trgt = int(edge["trgt"])
    l_z.append(sheet.vert_df.loc[srce, 'z'])
    l_rt.append(sheet.vert_df.loc[srce, 'rho']*atan2(sheet.vert_df.loc[srce, 'y'], sheet.vert_df.loc[srce, 'x']))
    l_z.append(sheet.vert_df.loc[trgt, 'z'])
    l_rt.append(sheet.vert_df.loc[trgt, 'rho']*atan2(sheet.vert_df.loc[trgt, 'y'], sheet.vert_df.loc[trgt, 'x']))



plt.plot(l_rt[0:2],l_z[0:2], 'gray')
for i in range(0,len(l_z),2):
    plt.plot(l_rt[i:i+2],l_z[i:i+2],'gray')
   
    

ax.autoscale()
ax.set_aspect("equal")
fig.set_size_inches(18.5, 10.5, forward=True)

plt.savefig('test_line.eps', dpi=True, transparent = True)

In [ ]:
from matplotlib.colors import ListedColormap
from tyssue.draw import sheet_view
def draw_specs(sheet):
    cmap = ListedColormap(["white", "yellow","red", 
                           "black","black","black"])
    specs = {
        'face': {
            'visible': True,
            'color': sheet.face_df.fold + sheet.face_df.is_mesoderm*3,
            'colormap':cmap,
            'alpha':0.7,
        },
        'edge': {
            'visible': True,
            'color':'black',
        },
        'vert':{
            'visible':False,
            's':100,
            'alpha': 0.3,
            'zorder': 2,
            'color': sheet.vert_df.radial_tension

        }

    }
    return specs


from polarity.toolbox import open_sheet,depth_calculation, all_depth_calculation
for p in list_perturb:
    sheet.face_df.loc[p, 'is_mesoderm']=1
sheet = sheet.extract_bounding_box(z_boundary=(-50,50))

sheet.vert_df['l_rt'] = sheet.vert_df['rho']*np.arctan2(sheet.vert_df['y'], sheet.vert_df['x'])
sheet.edge_df['dl_rt'] = np.arctan2(sheet.edge_df['dy'], sheet.edge_df['dx'])

fig, ax = sheet_view(sheet, coords=[ 'l_rt', 'z',], **draw_specs(sheet))
fig.set_size_inches(18.5, 10.5, forward=True)

plt.savefig('test_color.eps', dpi = 300, transparent=False)